# Kütüphanelerin import edilmesi

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score,classification_report,confusion_matrix
from lightgbm import LGBMClassifier
from sklearn.cluster import KMeans,SpectralClustering,AgglomerativeClustering
import matplotlib.pyplot as plt
import numpy as np
import csv

# Verinin okunması ve düzenlenmesi
Verideki boşluk karakterleri silinmiştir.
Veride not fire ve fire olmak üzere 2 sınıf bulunmaktadır. Bu sınıf bilgileri 0 ve 1 olarak değiştirilmiştir.

In [2]:
def trim(dataset):
    trim = lambda x: x.strip() if type(x) is str else x
    return dataset.applymap(trim)

data=trim(pd.read_csv("Algerian_forest_fires_dataset_UPDATE.csv", skiprows=[0, 124, 125, 126]))

data["Classes"]= data["Classes"].replace("not fire", 0)
data["Classes"]= data["Classes"].replace("fire", 1)

# Veri Hakkında Bilgiler
Veride eksik veri bulunmamaktadır. 13 adet özellik 1 adet sınıf kolonu bulunmaktadır.
Veriler; Cezayir'in kuzeydoğu kısmında yer alan Bejaia ve kuzeybatı kısmında yer alan Sidi Bel-abbes bölgelerinde meydana gelmiş orman yangını verilerinden elde edilmiştir.

1. Day : Gün
2. Month : Ay ('june' - 'september')
3. Year : Yıl : 2012

Hava durumu gözlemleri

4. Temperature : güneş tepedeyken maksimum sıcaklık (Celsius) : 22-42
5. RH : Bağıl Nem % : 21-90
6. Ws : Rüzgar Hızı km/saat : 6-29
7. Rain : Yağmur, total day in mm : 0-16.8

FWI Components

8. Fine Fuel Moisture Code (FFMC) index from the FWI system: 28.6 to 92.5
9. Duff Moisture Code (DMC) index from the FWI system: 1.1 to 65.9
10. Drought Code (DC) index from the FWI system: 7 to 220.4
11. Initial Spread Index (ISI) index from the FWI system: 0 to 18.5
12. Buildup Index (BUI) index from the FWI system: 1.1 to 68
13. Fire Weather Index (FWI) Index: Orman yangını riski, 0 to 31.1
14. Classes: tahmin edilecek kolon, fire / not fire (kategorik : 0/1)

In [3]:
print(data.isna().sum()) #Null değer var mı kontrolü

day            0
month          0
year           0
Temperature    0
 RH            0
 Ws            0
Rain           0
FFMC           0
DMC            0
DC             0
ISI            0
BUI            0
FWI            0
Classes        0
dtype: int64


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   day          244 non-null    int64  
 1   month        244 non-null    int64  
 2   year         244 non-null    int64  
 3   Temperature  244 non-null    int64  
 4    RH          244 non-null    int64  
 5    Ws          244 non-null    int64  
 6   Rain         244 non-null    float64
 7   FFMC         244 non-null    float64
 8   DMC          244 non-null    float64
 9   DC           244 non-null    float64
 10  ISI          244 non-null    float64
 11  BUI          244 non-null    float64
 12  FWI          244 non-null    float64
 13  Classes      244 non-null    int64  
dtypes: float64(7), int64(7)
memory usage: 26.8 KB


In [5]:
data["Classes"].value_counts() #Sınıfların dağılım bilgisi

1    138
0    106
Name: Classes, dtype: int64

In [6]:
data.corr() #Özellikler arası korelasyon

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes
day,1.000000,0.000000,NaN,0.095772,-0.074209,0.047001,-0.112265,0.224032,0.491571,0.527929,0.179301,0.517224,0.350234,0.201784
month,0.000000,1.000000,NaN,-0.059017,-0.037884,-0.041447,0.035322,0.015577,0.068178,0.127672,0.063545,0.085567,0.081732,0.022333
year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Temperature,0.095772,-0.059017,NaN,1.000000,-0.654443,-0.278132,-0.326786,0.677491,0.483105,0.370498,0.605971,0.456415,0.566839,0.518119
RH,-0.074209,-0.037884,NaN,-0.654443,1.000000,0.236084,0.222968,-0.645658,-0.405133,-0.220330,-0.688268,-0.349685,-0.580457,-0.435023
Ws,0.047001,-0.041447,NaN,-0.278132,0.236084,1.000000,0.170169,-0.163255,-0.001246,0.076245,0.012245,0.030303,0.033957,-0.066529
Rain,-0.112265,0.035322,NaN,-0.326786,0.222968,0.170169,1.000000,-0.544045,-0.288548,-0.296804,-0.347862,-0.299409,-0.324755,-0.379449
FFMC,0.224032,0.015577,NaN,0.677491,-0.645658,-0.163255,-0.544045,1.000000,0.602391,0.503910,0.740751,0.590251,0.691430,0.770114
DMC,0.491571,0.068178,NaN,0.483105,-0.405133,-0.001246,-0.288548,0.602391,1.000000,0.875358,0.678355,0.982206,0.875191,0.584188
DC,0.527929,0.127672,NaN,0.370498,-0.220330,0.076245,-0.296804,0.503910,0.875358,1.000000,0.503919,0.941672,0.737041,0.507122


# Aykırı değerleri temizleme
Her kolon için quantile alındı ve IQR değerine göre aykırı değerler atıldı.

In [7]:
columns=data.drop(["Classes","FWI"],axis=1).columns
Q1 = data[columns].quantile(0.25)
Q3 = data[columns].quantile(0.75)
IQR = Q3-Q1
print("Aykırı değer sayısı:"+str(len(data[((data[columns]<(Q1-1.5*IQR))|(data[columns]>(Q3+1.5*IQR))).any(axis=1)])))
data=data[~((data[columns]<(Q1-1.5*IQR))|(data[columns]>(Q3+1.5*IQR))).any(axis=1)]
data.reset_index(drop=True,inplace=True)

Aykırı değer sayısı:68


# Veri, özellikler ve sınıflar olmak üzere 2 parçaya bölündü.

In [8]:
X=data.drop(["Classes","FWI"],axis=1)
y=data["Classes"]

# Özellik Ayırt Ediciliği

In [9]:
xgb=RandomForestClassifier().fit(X,y)
plt.bar(X.columns, xgb.feature_importances_)

<BarContainer object of 12 artists>

Day, month, year ve Ws kolonlarının ayırt ediciliği olmadığı için silinmiştir.

In [10]:
X=X.drop(["day", "year","month", " Ws"],axis=1)

Belirtilen kolonlar silindikten sonra özellik ayırt ediciliği aşağıdaki gibi olmuştur.

In [11]:
xgb=RandomForestClassifier().fit(X,y)
plt.bar(X.columns, xgb.feature_importances_)

<BarContainer object of 8 artists>

# Normalizasyon
Normalizasyon için Standart Scaler yöntemi kullanılmıştır.

In [12]:
scaler=StandardScaler()
scaler.fit(X)
X_normalized=scaler.transform(X)

# Train Test Ayırma

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Sınıflandırma 

# XGBoost

In [14]:
xgb=XGBClassifier(n_jobs=-1)
xgb.fit(X_train,y_train)
predicted=xgb.predict(X_test)

[17:22:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


D:\Programlar\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [15]:
print(classification_report(y_test, predicted, digits=3))

              precision    recall  f1-score   support

           0      1.000     1.000     1.000        11
           1      1.000     1.000     1.000        25

    accuracy                          1.000        36
   macro avg      1.000     1.000     1.000        36
weighted avg      1.000     1.000     1.000        36



In [16]:
print(confusion_matrix(y_test, predicted))

[[11  0]
 [ 0 25]]


# LightGBM

In [17]:
lgb=LGBMClassifier(n_jobs=-1)
lgb.fit(X_train,y_train)
predicted=lgb.predict(X_test)

In [18]:
print(classification_report(y_test, predicted, digits=3))

              precision    recall  f1-score   support

           0      1.000     0.909     0.952        11
           1      0.962     1.000     0.980        25

    accuracy                          0.972        36
   macro avg      0.981     0.955     0.966        36
weighted avg      0.973     0.972     0.972        36



In [19]:
print(confusion_matrix(y_test, predicted))

[[10  1]
 [ 0 25]]


# RandomForest

In [20]:
rfc=RandomForestClassifier(n_jobs=-1)
rfc.fit(X_train,y_train)
predicted=rfc.predict(X_test)

In [21]:
print(classification_report(y_test, predicted, digits=3))

              precision    recall  f1-score   support

           0      1.000     1.000     1.000        11
           1      1.000     1.000     1.000        25

    accuracy                          1.000        36
   macro avg      1.000     1.000     1.000        36
weighted avg      1.000     1.000     1.000        36



In [22]:
print(confusion_matrix(y_test, predicted))

[[11  0]
 [ 0 25]]


# Sınıflandırma Karşılaştırma
Yapılan çalışma sonucunda en yüksek macro f1 skor Xgboost ve Random Forest algoritmaları ile elde edilmiştir. Ardından Lightgbm algoritması gelmiştir.

1. Xgboost 100 Macro F1

2. Lightgbm 96.8 Macro F1

3. Random Forest 100 Macro F1

# Kümeleme

# Spectral Clustering

In [23]:
sc = SpectralClustering(n_clusters=2,assign_labels="discretize")
predicted=sc.fit_predict(X_normalized)

# Sınıf Etiketlerinin Belirlenmesi
Bu kısımda modelden elde edilen etiketler olması gereken etiketler ile kontrol edilmiştir. Yanlış etiket sayısı doğru etiket sayısından fazla ise etiketler yer değiştirilmektedir.

In [24]:
dogru=0
yanlis=0
for i in range(len(y)):
    if(y[i]==0):
        if(predicted[i]==0):
            dogru+=1
        else:
            yanlis+=1        
if(yanlis>dogru):
    zeros=np.where(predicted==0)
    ones=np.where(predicted==1)
    predicted[zeros]=1
    predicted[ones]=0

In [25]:
print(classification_report(y, predicted, digits=3))

              precision    recall  f1-score   support

           0      0.252     0.603     0.355        63
           1      0.000     0.000     0.000       113

    accuracy                          0.216       176
   macro avg      0.126     0.302     0.178       176
weighted avg      0.090     0.216     0.127       176



# KMeans Clustering

In [26]:
kmeans = KMeans(n_clusters=2)
predicted=kmeans.fit_predict(X_normalized)

# Sınıf Etiketlerinin Belirlenmesi
Bu kısımda modelden elde edilen etiketler olması gereken etiketler ile kontrol edilmiştir. Yanlış etiket sayısı doğru etiket sayısından fazla ise etiketler yer değiştirilmektedir.

In [27]:
dogru=0
yanlis=0
for i in range(len(y)):
    if(y[i]==0):
        if(predicted[i]==0):
            dogru+=1
        else:
            yanlis+=1        
if(yanlis>dogru):
    zeros=np.where(predicted==0)
    ones=np.where(predicted==1)
    predicted[zeros]=1
    predicted[ones]=0

In [28]:
print(classification_report(y, predicted, digits=3))

              precision    recall  f1-score   support

           0      0.652     0.952     0.774        63
           1      0.964     0.717     0.822       113

    accuracy                          0.801       176
   macro avg      0.808     0.835     0.798       176
weighted avg      0.853     0.801     0.805       176



# Agglomerative Clustering

In [29]:
ac = AgglomerativeClustering(n_clusters=2)
predicted=ac.fit_predict(X_normalized)

# Sınıf Etiketlerinin Belirlenmesi
Bu kısımda modelden elde edilen etiketler olması gereken etiketler ile kontrol edilmiştir. Yanlış etiket sayısı doğru etiket sayısından fazla ise etiketler yer değiştirilmektedir.

In [30]:
dogru=0
yanlis=0
for i in range(len(y)):
    if(y[i]==0):
        if(predicted[i]==0):
            dogru+=1
        else:
            yanlis+=1        
if(yanlis>dogru):
    zeros=np.where(predicted==0)
    ones=np.where(predicted==1)
    predicted[zeros]=1
    predicted[ones]=0

In [31]:
print(classification_report(y, predicted, digits=3))

              precision    recall  f1-score   support

           0      0.860     0.587     0.698        63
           1      0.805     0.947     0.870       113

    accuracy                          0.818       176
   macro avg      0.832     0.767     0.784       176
weighted avg      0.825     0.818     0.808       176



# Kümeleme Karşılaştırma
Yapılan çalışma sonucunda en yüksek macro f1 skor KMeans Clustering algoritması ile elde edilmiştir. Ardından Agglomerative Clustering ve Spectral Clustering algoritmaları gelmiştir.

1. Spectral Clustering 17.8 Macro F1

2. KMeans Ckustering 79.8 Macro F1

3. Agglomerative Clustering 78.4 Macro F1

# Contributors

16011010 - Furkan Necati Ürkmez

18011801 - Mustafa Şentürk

17011025 - Furkan Akçakaya